In [176]:
import pandas as pd #Importar principais bibliotecas para auxiliar na leitura, manipulação, tratamento e análise dos dados
from sklearn.model_selection import train_test_split #Importa a ferramenta para realizar treinamento e teste nos modelos
from sklearn.ensemble import RandomForestClassifier #Importa a ferramente para realizar predições árvore de decisão
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix #Importa as ferramentas de metrica de precisão
from sklearn.preprocessing import LabelEncoder #Importa a ferramenta para converter rótulos de classe (categorias) em números.

In [177]:
dados = pd.read_csv('Hopsital Dataset.csv') #Usa biblioteca pandas para abrir arquivo data.csv, que contem os dados a serem analisados,
#https://www.kaggle.com/datasets/minsithu/hospital-antibiotics-usage/

In [178]:
dados.head(5) #Verificar se o arquivo a ser analisado esta correto. Podendo ser visualizado os dados iniciais

,Age,Date of Data Entry,Gender,Diagnosis,Name of Drug,Dosage (gram),Route,Frequency,Duration (days),Indication
0,85,19/12/2019 14:41:49,Female,"ccf, hypertension, ida, ckd(stage 5), ?icm,",ceftriaxone,1,IV,BD,7,icm
1,87,19/12/2019 16:35:25,Female,"pad(lt u.l), be amputation,/post op, akt",ceftriaxone,1,IV,BD,1,post op
2,82,19/12/2019 15:48:49,Male,"type-2dm, ihd, col, copd, ht",ofloxacin,0.4,IV,BD,3,abd distension with leg swelling
3,82,19/12/2019 15:50:33,Male,"type-2 dm, ihd, col, copd, ht",cefipime,1,IV,BD,5,abd distension with leg swelling
4,82,19/12/2019 15:52:20,Male,"type-2 dm, ihd, col, copd, ht",azithromycin,0.5,Oral,OD,3,abd distension with leg swelling


In [179]:
dados.info()  #Verifica se o tipo de dados estão corretos e se existe alguma dado faltando

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 833 entries, 0 to 832
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   Age                 833 non-null    object
 1   Date of Data Entry  833 non-null    object
 2   Gender              833 non-null    object
 3   Diagnosis           833 non-null    object
 4   Name of Drug        833 non-null    object
 5   Dosage (gram)       833 non-null    object
 6   Route               833 non-null    object
 7   Frequency           833 non-null    object
 8   Duration (days)     833 non-null    object
 9   Indication          832 non-null    object
dtypes: object(10)
memory usage: 65.2+ KB


In [180]:
dados.isna().sum() #Verificar e contar dados nulos em todas as colunas.

Age                   0
Date of Data Entry    0
Gender                0
Diagnosis             0
Name of Drug          0
Dosage (gram)         0
Route                 0
Frequency             0
Duration (days)       0
Indication            1
dtype: int64

In [181]:
dados.isna().any() #Outro forma de ver se há dados faltantes

Age                   False
Date of Data Entry    False
Gender                False
Diagnosis             False
Name of Drug          False
Dosage (gram)         False
Route                 False
Frequency             False
Duration (days)       False
Indication             True
dtype: bool

In [182]:
dados[pd.isnull(dados['Indication'])] #Mostra a linha que há dado ausente

,Age,Date of Data Entry,Gender,Diagnosis,Name of Drug,Dosage (gram),Route,Frequency,Duration (days),Indication
832,85,19/12/2019 14:41:49,Female,"ccf, hypertension, ida, ckd(stage 5), ?icm,",ceftriaxone,1,IV,BD,7,NaN


In [183]:
dados['Indication'] = dados['Indication'].fillna('icm') #Opto por substituir o dado faltante por um diagnóstico compativel

In [184]:
dados.duplicated().sum()#Identificar se existe linhas duplicadas

8

In [185]:
duplicate = dados[dados.duplicated()] #Mostrar quais dados então duplicados
duplicate

,Age,Date of Data Entry,Gender,Diagnosis,Name of Drug,Dosage (gram),Route,Frequency,Duration (days),Indication
157,85,19/12/2019 14:41:49,Female,"ccf, hypertension, ida, ckd(stage 5), ?icm,",ceftriaxone,1,IV,BD,7,ccf
274,85,19/12/2019 14:41:49,Female,"ccf, hypertension, ida, ckd(stage 5), ?icm,",ceftriaxone,1,IV,BD,7,ckd
465,18,19/12/2019 14:51:39,Male,"hepatitis b,",amoxicillin+flucloxacillin,1,Oral,TDS,5,hepatitis
494,85,19/12/2019 14:41:49,Female,"ccf, hypertension, ida, ckd(stage 5), ?icm,",ceftriaxone,1,IV,BD,7,hypertension
507,85,19/12/2019 14:41:49,Female,"ccf, hypertension, ida, ckd(stage 5), ?icm,",ceftriaxone,1,IV,BD,7,icm
509,85,19/12/2019 14:41:49,Female,"ccf, hypertension, ida, ckd(stage 5), ?icm,",ceftriaxone,1,IV,BD,7,ida
703,31,19/12/2019 14:30:33,Female,"rvi, stage4, art noise, hepatitis, bone marrow...",cefipime,1,IV,BD,4,rvi
832,85,19/12/2019 14:41:49,Female,"ccf, hypertension, ida, ckd(stage 5), ?icm,",ceftriaxone,1,IV,BD,7,icm


In [186]:
dados = dados.drop_duplicates() #Elimina os dados duplicados

In [187]:
dados['Name of Drug'].unique() #Na coluna 'Name of Drug' mostra todos os antibióticos usados

array(['ceftriaxone', 'ofloxacin', 'cefipime', 'azithromycin',
       'ceftazidime', 'septrin', 'co-amoxiclav', 'clindamycin',
       'cefoperazone+sulbactam', 'metronidazole', 'cefixime', 'cefepime',
       'ciprofloxacin', 'norfloxacin', 'coamoxiclav', 'cifran',
       'meropenem', 'gentamicin', 'pen v', 'clarithromycin', 'mirox',
       'amoxicillin', 'cefexime', 'amikacin', 'ceftazidine', 'rifaximin',
       'cefoperazone', 'levofloxacin', 'amoxicillin+flucloxacillin',
       'linezolid', 'ceftriaxone+sulbactam', 'amoxicillin+flucoxiacillin',
       'clarthromycin', 'rifampicin', 'streptomycin',
       'amoxicillin+flucoxacillin', 'cefoparazone+sulbactam',
       'flucloxacillin', 'imipenem', 'doxycycline', 'amoxiclav',
       'amoxicillin+flucloaxcin', 'dazolic', 'cefaziclime', 'ceftiaxone',
       'cefteiaxone', 'Drugs', 'Name of Drug', 'vancomycin',
       'pentoxyfylline', 'doxycyclin', 'levefloxacin', 'menopem',
       'pentoxifylline', 'amoxicillin+flucloxiacillin',
       'p

In [188]:
dados['Name of Drug'].value_counts() #Conta quantas vezes o antibiótico aparece na coluna

ceftriaxone                    215
co-amoxiclav                   162
metronidazole                   59
cefixime                        58
septrin                         37
clarithromycin                  32
levofloxacin                    31
amoxicillin+flucloxacillin      28
ceftazidime                     24
cefepime                        14
cefipime                        12
clindamycin                     12
rifaximin                       10
coamoxiclav                      9
cefoperazone                     9
amikacin                         9
meropenem                        8
ciprofloxacin                    7
rifampicin                       5
gentamicin                       5
pen v                            5
azithromycin                     5
ceftazidine                      4
streptomycin                     4
clarthromycin                    4
mirox                            4
cifran                           4
amoxicillin                      4
cefoparazone+sulbact

In [189]:
dados = dados.drop('Date of Data Entry',axis=1) #Elimina a coluna 'Date of Data Entry' não tem relevancia no momento

In [190]:
dados['Age'] = pd.to_numeric(dados['Age'], errors='coerce') #Transforma os dados da coluna 'Age' em valores numéricos

In [191]:
# Exibir as linhas com valores nulos
dados[dados.isnull().any(axis=1)]
#Podemos observar que no meio do dataset há uma linha repetida com o titulo das colunas

,Age,Gender,Diagnosis,Name of Drug,Dosage (gram),Route,Frequency,Duration (days),Indication
529,NaN,Sex,Diagnosis,Name of Drug,Dosage (gram),Route,Frequency,Duration (days),Indication


In [192]:
dados.dropna(subset=['Age'],axis=0,how='any',inplace=True) #Excluo linha com valor nulo. Ou seja linha com titulo repetido

In [193]:
dados.info() #Ver quais são os tipos do dados e se estão corretos

<class 'pandas.core.frame.DataFrame'>
Int64Index: 824 entries, 0 to 831
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Age              824 non-null    float64
 1   Gender           824 non-null    object 
 2   Diagnosis        824 non-null    object 
 3   Name of Drug     824 non-null    object 
 4   Dosage (gram)    824 non-null    object 
 5   Route            824 non-null    object 
 6   Frequency        824 non-null    object 
 7   Duration (days)  824 non-null    object 
 8   Indication       824 non-null    object 
dtypes: float64(1), object(8)
memory usage: 64.4+ KB


In [194]:
dados['Dosage (gram)'] = pd.to_numeric(dados['Dosage (gram)'], errors='coerce') #Transforma os dados da coluna 'Dosagem' em valores numéricos

In [195]:
# Exibir as linhas com valores nulos
dados[dados.isnull().any(axis=1)]

,Age,Gender,Diagnosis,Name of Drug,Dosage (gram),Route,Frequency,Duration (days),Indication
528,82.0,Sex,Diagnosis,Drugs,NaN,Route,Freq,Duration,Indication


In [196]:
dados = dados.drop(528) #Excluo a linha pelo indice

In [197]:
dados_tratados = dados

In [198]:
dados['Diagnosis'].head() #Mostra os dados da coluna 'Diagnosis'

0    ccf, hypertension, ida, ckd(stage 5), ?icm,
1       pad(lt u.l), be amputation,/post op, akt
2                   type-2dm, ihd, col, copd, ht
3                  type-2 dm, ihd, col, copd, ht
4                  type-2 dm, ihd, col, copd, ht
Name: Diagnosis, dtype: object

In [199]:
dados['Diagnosis'] = dados['Diagnosis'].str.split(',') #Separa a lista de diagósticos pela virgula

In [200]:
dados = dados.explode('Diagnosis') #Para cada diagnóstico cria uma linha

In [201]:
dados.head(5) #Mosta como esta parte dos dados até o momento

,Age,Gender,Diagnosis,Name of Drug,Dosage (gram),Route,Frequency,Duration (days),Indication
0,85.0,Female,ccf,ceftriaxone,1.0,IV,BD,7,icm
0,85.0,Female,hypertension,ceftriaxone,1.0,IV,BD,7,icm
0,85.0,Female,ida,ceftriaxone,1.0,IV,BD,7,icm
0,85.0,Female,ckd(stage 5),ceftriaxone,1.0,IV,BD,7,icm
0,85.0,Female,?icm,ceftriaxone,1.0,IV,BD,7,icm


In [202]:
dados['Diagnosis'] = dados['Diagnosis'].str.upper().str.strip() #Deixa todos os diagósticos em maiusculo e tira excesso de espaço do começo e do final
#(Havia diagnósticos repetidos por esta em maiusculo e minusculo e outros com excesso de espaço)

In [203]:
dados['Diagnosis'].nunique() #Faz a contagem dos diagósticos unicos

376

In [204]:
#Mostra todos os diagnósticos em ordem alfabética
ordem = dados['Diagnosis'].unique()
ordem.sort()
ordem

array(['', '/POST OP', '1ST LINE ART RESISTENCE', '?ICM', 'ABD TB',
       'ABSCESS ON CHIN', 'ACUTE BACILLARY DYSENTERY E SEPSIS',
       'ACUTE BACILLARY DYSENTRY E SEPSIS', 'ACUTE BRONCHITIS',
       'ACUTE CEREBRAL INFARCT', 'ACUTE COPD',
       'ACUTE EXACERBATION OF COPD', 'ACUTE FLACCID LOWER LIMB WEAKNESS',
       'ACUTE GASTRIC EROSION', 'ACUTE GASTRITIS VOMITTING Ē HK+',
       'ACUTE GE', 'ACUTE HEPATITIS', 'ACUTE LIVER DISEASE',
       'ACUTE ON CHRONIC HEART FAILURE', 'ACUTE PULMONARY EDEMA',
       'ACUTE SEVER ASTHMA', 'ACUTE SEVERE ASTHMA', "ADDISION'S DISEASE",
       "ADDISON'S DISEASE", 'AF', 'AF Ē RVR', 'AGE', 'AKI', 'AKT',
       'ALCOHOL COL', 'ALCOHOL HEPATITIS', 'ALCOHOL WITHDRAWAL',
       'ALCOHOL WITHDRAWAL SYNDROME', 'ALCOHOLIC  COL', 'ALCOHOLIC COL',
       'ALCOHOLIC GASTRITIS', 'ALCOHOLIC HEPATITIS',
       'ALCOHOLIC WITHDRAWAL', 'ANAEMIA', 'ANAPHYATIC',
       'ANAPHYLATIC SHOCK', 'ANAPHYLAXIS (?PENICILLIN)', 'AND TB',
       'ANTEROSEPTAL MI', 'ANTI TB

In [205]:
le = LabelEncoder() #Realiza transformaçao dos dados categoricos em numéricos
dados['Gender'] = le.fit_transform(dados['Gender'])
dados['Diagnosis'] = le.fit_transform(dados['Diagnosis'])
dados['Name of Drug'] = le.fit_transform(dados['Name of Drug'])
dados['Route'] = le.fit_transform(dados['Route'])
dados['Frequency'] = le.fit_transform(dados['Frequency'])
dados['Indication'] = le.fit_transform(dados['Indication'])

In [206]:
X = dados.drop('Name of Drug', axis=1) # Faz separação de traino e teste de um único dataframe. Como separei usar secula anterior
y = dados['Name of Drug']

In [207]:
# Separa os dados em treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Inicializar o classificador (Random Forest Classifier neste exemplo)
clf = RandomForestClassifier(random_state=42) #após teste agoritimo se mostrou melhor em prever

# Treinar o modelo
clf.fit(X_train, y_train)

# Fazer previsões no conjunto de teste
y_pred = clf.predict(X_test)

# Avaliar o desempenho do modelo
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

# Exibir outras métricas de avaliação
print(classification_report(y_test, y_pred))

# Exibir a matriz de confusão
conf_matrix = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:")
print(conf_matrix)

Accuracy: 0.9712643678160919
              precision    recall  f1-score   support

           0       1.00      1.00      1.00         4
           1       0.50      0.50      0.50         2
           2       0.00      0.00      0.00         1
           3       0.67      0.67      0.67         6
           4       0.00      0.00      0.00         1
           5       1.00      1.00      1.00         3
           7       1.00      1.00      1.00         1
           8       1.00      0.75      0.86         4
           9       1.00      1.00      1.00         1
          10       1.00      1.00      1.00         8
          12       1.00      0.93      0.96        14
          13       1.00      1.00      1.00        22
          14       1.00      1.00      1.00         3
          15       1.00      1.00      1.00         5
          17       1.00      1.00      1.00        12
          18       1.00      1.00      1.00         1
          19       0.00      0.00      0.00         

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# Experimento

A coluna 'Diagnosis' apresenta diagnósticos sem relação direta com o uso do antibiótico. Como teste optei por retirar essa coluna para avaliar se essa produzia ruido nos dados ou apresentava dados complementares para a previsão.

In [208]:
dados = dados_tratados
dados.head() #Recupera os dados tratados inicialmente

,Age,Gender,Diagnosis,Name of Drug,Dosage (gram),Route,Frequency,Duration (days),Indication
0,85.0,Female,"[ccf, hypertension, ida, ckd(stage 5), ?ic...",ceftriaxone,1.0,IV,BD,7,icm
1,87.0,Female,"[pad(lt u.l), be amputation, /post op, akt]",ceftriaxone,1.0,IV,BD,1,post op
2,82.0,Male,"[type-2dm, ihd, col, copd, ht]",ofloxacin,0.4,IV,BD,3,abd distension with leg swelling
3,82.0,Male,"[type-2 dm, ihd, col, copd, ht]",cefipime,1.0,IV,BD,5,abd distension with leg swelling
4,82.0,Male,"[type-2 dm, ihd, col, copd, ht]",azithromycin,0.5,Oral,OD,3,abd distension with leg swelling


In [209]:
dados = dados.drop('Diagnosis',axis=1)
dados.head()

,Age,Gender,Name of Drug,Dosage (gram),Route,Frequency,Duration (days),Indication
0,85.0,Female,ceftriaxone,1.0,IV,BD,7,icm
1,87.0,Female,ceftriaxone,1.0,IV,BD,1,post op
2,82.0,Male,ofloxacin,0.4,IV,BD,3,abd distension with leg swelling
3,82.0,Male,cefipime,1.0,IV,BD,5,abd distension with leg swelling
4,82.0,Male,azithromycin,0.5,Oral,OD,3,abd distension with leg swelling


In [210]:
le = LabelEncoder() #Realiza transformaçao dos dados categoricos em numéricos
dados['Gender'] = le.fit_transform(dados['Gender'])
dados['Name of Drug'] = le.fit_transform(dados['Name of Drug'])
dados['Route'] = le.fit_transform(dados['Route'])
dados['Frequency'] = le.fit_transform(dados['Frequency'])
dados['Indication'] = le.fit_transform(dados['Indication'])

In [211]:
X = dados.drop('Name of Drug', axis=1) # Faz separação de traino e teste de um único dataframe. Como separei usar secula anterior
y = dados['Name of Drug']

In [212]:
# Separa os dados em treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Inicializar o classificador (Random Forest Classifier neste exemplo)
clf = RandomForestClassifier(random_state=42) #após teste agoritimo se mostrou melhor em prever

# Treinar o modelo
clf.fit(X_train, y_train)

# Fazer previsões no conjunto de teste
y_pred = clf.predict(X_test)

# Avaliar o desempenho do modelo
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

# Exibir outras métricas de avaliação
print(classification_report(y_test, y_pred))

# Exibir a matriz de confusão
conf_matrix = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:")
print(conf_matrix)

Accuracy: 0.806060606060606
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         1
           1       0.00      0.00      0.00         3
           3       0.50      0.60      0.55         5
           5       0.00      0.00      0.00         1
           6       0.00      0.00      0.00         1
           8       0.50      1.00      0.67         1
           9       0.00      0.00      0.00         2
          10       0.00      0.00      0.00         6
          11       1.00      1.00      1.00         1
          12       1.00      1.00      1.00         1
          13       0.83      1.00      0.91        10
          14       1.00      1.00      1.00         1
          15       1.00      1.00      1.00         2
          16       0.00      0.00      0.00         1
          17       0.57      0.80      0.67         5
          18       0.00      0.00      0.00         0
          20       0.00      0.00      0.00         1

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.

Observa-se com isso que até mesmo dados que muitas vezes podem parecer irrelevantes são primordiais na previsão. Com dados que inicialmente achavasse serem irrelevantes e ruidosos acrescentou-se uma porcentagem de mais 17% na correta previsão, indo de 80%, sem a coluna Diagnosis, para 97% com essa coluna.